In [5]:
from langchain_experimental.agents import create_csv_agent
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
load_dotenv()
import pandas as pd

In [4]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [8]:
agent1 = create_csv_agent(
            llm, "mimic/hosp/admissions.csv", verbose=True)

In [9]:
agent2 = create_csv_agent(
            llm, "mimic/hosp/", verbose=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: To find the number of unique patients in the dataframe, I need to count the unique values in the `subject_id` column. The pandas function `nunique()` can be used to achieve this.
Action: python_repl_ast
Action Input: df['subject_id'].nunique()
Observation: 100
Thought:I now know the final answer
Final Answer: There are 100 unique patients.

> Finished chain.


'There are 100 unique patients.'

In [24]:
# import requests
# import time
# # Define the API endpoint and parameters
# url = "https://api-ares.traversaal.ai/live/predict"
# api_key = "ares_644291f1c734c64a8c586e4b8b069b533390dd46d6fcee0c438f11528b4d8f8a"
# headers = {"x-api-key": api_key, "content-type": "application/json"}
# query = "I want to eat best food san francisco, give me some suggestions"
# api_parameters = {"query": query}
# # Print out the request headers before sending the request
# print("Request Headers:")
# print(headers)
# # Retry the API request if a connection error occurs
# max_retries = 7
# retry_delay = 4
# #api_data={}
# for retry in range(max_retries):
#     try:
#         # Send a request to the API
#         api_response = requests.get(url, json=api_parameters, headers=headers)
#         # Extract the API response data
#         api_data = api_response.json()
#         break
#     except requests.exceptions.ConnectionError:
#         print(f"Connection error occurred. Retrying in {retry_delay} seconds...")
#         time.sleep(retry_delay)
# else:
#     print("Max retries exceeded. Unable to establish a connection to the API.")

# # Create the RAG prompt
# prompt = "I am performing RAG (Retrieve, Augment, Generate) using an API. Here is the context:\n\n"

# print(api_data)

# # # Iterate over the API data and generate responses for each query
# # for item in api_data["data"]:
# #     prompt += f"{item['context']}\n\n"
# #     prompt += f"Query: {item['query']}\n\n"
# #     prompt += "Generate a response using the provided context:\n\n"

# #     # Generate a response using the RAG model
# #     response = llm.invoke(prompt)
# #     # Extract the generated response
# #     generated_response = response

# #     # Print the generated response
# #     print(generated_response)

Request Headers:
{'x-api-key': 'ares_644291f1c734c64a8c586e4b8b069b533390dd46d6fcee0c438f11528b4d8f8a', 'content-type': 'application/json'}
{'message': 'Missing Authentication Token'}


In [129]:
import requests

url = "https://api-ares.traversaal.ai/live/predict"
query = "pizza spots in San Francisco"
payload = {
    "query": query,
}
headers = {
  "x-api-key": "ares_a0866ad7d71d2e895c5e05dce656704a9e29ad37860912ad6a45a4e3e6c399b5",
  "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [130]:
response_data = response.json()
print(len(response_data))

1


In [131]:
print(response_data)

{'error': 'Application Internal Server Error. Contact support '}


In [132]:
response_text = response_data['data']['response_text']
web_urls = response_data['data']['web_url']

KeyError: 'data'

In [133]:
print(response_text)

1. What is the best pizzeria in Italy according to the 50 Top Pizza guide?
- Pizzeria I Masanielli in Caserta is ranked as the best pizzeria in Italy by the 50 Top Pizza guide.

2. Where is Pizzeria I Masanielli located?
- Pizzeria I Masanielli is located in Caserta, a small city about 40 minutes north of Naples in Italy.

3. What are some of the standout pizzas at Pizzeria I Masanielli?
- Some standout pizzas at Pizzeria I Masanielli include "Le 5 Consistenze della Cipolla" topped with fior di latte cheese and onions in five consistencies, and the "Genovese Secondo Martucci" with stewed onions, fior di latte, pork cheek, and Roman ricotta conciato cheese.

4. How can you make a reservation at Pizzeria I Masanielli?
- Reservations can be made online through the restaurant's reservation system. It is recommended to book a table two months in advance.

5. What is the atmosphere like at Pizzeria I Masanielli?
- Pizzeria I Masanielli is described as busy and crowded, with a lively atmosphe

In [ ]:
print(web_urls)

['https://www.foodandwine.com/travel/europe/italy/pizzas-in-italy', 'https://www.cntraveler.com/galleries/2014-12-01/10-best-pizzerias-in-italy-rome-naples-venice', 'https://www.blueroadstouring.com/blog/uncover-best-pizza-italy', 'https://www.reddit.com/r/askitaly/comments/wvxl8t/best_pizza_in_italyromemilanvenice/', 'https://community.ricksteves.com/travel-forum/italy/best-pizza-in-italy-map-it', 'https://www.timeout.com/news/its-official-these-are-the-best-pizzerias-in-italy-right-now-071323', 'https://www.foodandwine.com/best-pizzeria-world-7106600', 'https://www.enjoytravel.com/en/travel-news/food-and-drink/pizza-in-italy', 'https://theromanguy.com/italy-travel-blog/italy-travel-blog/italy/the-ultimate-guide-to-pizza-in-italy-eat-pizza-like-a-local/', 'https://www.thedailymeal.com/travel/15-best-pizza-places-italy-slideshow/']


In [138]:
prompt = ""
prompt += f"{response_text}\n\n"
prompt += f"Query: Pizza spots in Italy\n\n"
prompt += "Generate a response using the provided context:\n\n"

# Generate a response using the RAG model
response = llm.invoke(prompt)
# Extract the generated response
generated_response = response

# Print the generated response
print(generated_response)

content="There are several notable pizza spots in Italy. The best pizzeria, according to the 50 Top Pizza guide, is Pizzeria I Masanielli located in Caserta, which is known for its high-quality ingredients and gourmet approach to pizza-making. Other significant pizza places include Di Matteo and Pizzeria da Albert in Naples, Ai Marmi in Rome, Mamma Rosa in Marche, Pizzeria da Michele in Naples, Gino Sorbillo in Naples and Milan, Pizzeria Santarpia in Florence, Pizzeria Port'Alba in Naples, Da Remo in Rome, Antico Forno in Venice, Nicola's in Naples, Spontini in Milan, and Pepe in Grani in Caserta. Each of these pizzerias has its own specialty and unique style of pizza-making that contributes to the diverse pizza culture in Italy."
